#### Calculating time taken by the program

In [1]:
from datetime import datetime as dt

ct = dt.now()
print(ct)
# Code here
print(dt.now() - ct)

2019-04-01 21:45:37.166605
0:00:00.001608


In [9]:
import time

ct = time.process_time_ns()
for _ in range(1000000):
    pass
print((time.process_time_ns() - ct))


44450775


In [8]:
import timeit

pre = '''import time'''
code = '''
for i in range(10**6):
    pass

'''
timeit.timeit(setup=pre, stmt=code, number=20, globals=globals())

0.42006790500090574

In [10]:
import time
code = '''
for i in range(10**6):
    pass
'''
timeit.timeit(stmt=code, number=3, timer=time.process_time_ns)

82449098

#### Write a decorator that shows the nanoseconds taken by a fuction to run

In [25]:
import time
def deco(func):
    def wrap():
        ct=time.process_time_ns()
        func()
        print("Time of execution =",time.process_time_ns()-ct)
    return wrap

@deco
def my_func():
    for i in range(100):
        pass
    print("my_func executed")
my_func()

my_func executed
Time of execution = 862784


In [2]:
import cProfile

def add(a, b):
    for _ in range(1000000):
        pass
    return a + b

prof = cProfile.Profile()
prof.enable()
for _ in range(5):
    [add(i, i*2) for i in range(10)]
prof.disable()
prof.print_stats()

         76 function calls in 1.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.000    1.000 <ipython-input-2-9d6837888748>:10(<module>)
        5    0.000    0.000    0.999    0.200 <ipython-input-2-9d6837888748>:11(<listcomp>)
        1    0.000    0.000    0.000    0.000 <ipython-input-2-9d6837888748>:12(<module>)
       50    0.999    0.020    0.999    0.020 <ipython-input-2-9d6837888748>:3(add)
        2    0.000    0.000    0.000    0.000 codeop.py:132(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:142(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:207(pre_run_code_hook)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:116(<lambda>)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:1266(user_global_ns)
        2    0.000    0.000    1.000    0.500 interactiveshell.py:3254(run_code)
        2    0.000    0.000    0

## Memoization
###### Memoization allows you to optimize a function by caching its output based on the parameters you supply to it. Once you memoize a function, it will only compute its output once for each set of parameters you call it with. Every call after the first will be quickly retrieved from a cache.

In [4]:
import timeit

def fibonacci(k):
   if k==0:
    return 0
   if k==1:
    return 1
   else:
    return fibonacci(k-1) + fibonacci(k-2)


In [10]:
timeit.timeit(stmt ='fibonacci(35)', number = 1, globals = globals())

3.4305048840033123

### Writing a function decorator for memoization 

In [11]:
def memoization(func):
    cache = {}
    def wrapper(*args):
        if args in cache:
            return cache[args]
        else:
            res = func(*args)
            cache[args] = res
            return res
    return wrapper

In [15]:
# Running fibonacci(30) for the first time
memoized = memoization(fibonacci)

print(timeit.timeit('memoized(25)', globals = globals(), number = 1))

print(timeit.timeit('memoized(30)', globals = globals(), number = 1))
print(memoized.__closure__[0].cell_contents)

0.045252593998156954
0.327621447002457
{(25,): 75025, (30,): 832040}


In [16]:
print(memoized.__closure__[1].cell_contents)

<function fibonacci at 0x7ff005155c80>


### Using a built-in decorator for memoization

In [17]:
from functools import lru_cache

@lru_cache(maxsize = 512)
def fibonacci(k):
   if k==0:
    return 0
   if k==1:
    return 1
   else:
    return fibonacci(k-1) + fibonacci(k-2)

print(timeit.timeit('fibonacci(10)', globals = globals(), number = 1))

print(timeit.timeit('fibonacci(75)', globals = globals(), number = 1))

print(timeit.timeit('fibonacci(150)', globals = globals(), number = 1))

print(timeit.timeit('fibonacci(500)', globals = globals(), number = 1))

fibonacci.cache_info()

1.5060002624522895e-05
7.932799780974165e-05
8.291999984066933e-05
0.0005404839976108633


CacheInfo(hits=501, misses=501, maxsize=512, currsize=501)

### Where to use Memoization ?

### namedtuples
###### namedtuples are just like tuples with the difference being you can access the data using indexes or field names
###### namedtuple are immutable just like tuples

In [20]:
from collections import namedtuple
Student = namedtuple('Stu',['grade', 'roll_no'])

In [21]:
stud_1 = Student(9.8,1145)
print(stud_1.grade, stud_1[0], getattr(stud_1, 'grade'))

9.8 9.8 9.8


In [23]:
stud_1.grade = 9.5

AttributeError: can't set attribute

#### Built-in methods for namedtuple
###### _asdict() - Returns the namedtuple as an OrderedDict

In [24]:
stud_1._asdict()

OrderedDict([('grade', 9.8), ('roll_no', 1145)])

###### _fields() - Returns all the fields of the namedtuple

In [25]:
stud_1._fields

('grade', 'roll_no')

###### _replace() - Replaces the value of a given field (Only way to change values in namedtuple)

In [26]:
stud_1._replace(roll_no= 1000)

Stu(grade=9.8, roll_no=1000)

In [36]:
stud_1

Stu(grade=9.8, roll_no=1145)

###### _make() - Returns a namedtuple from a iterable

In [27]:
values = [8.5, 1455]
stud_2 = Student._make(values)
stud_2

Stu(grade=8.5, roll_no=1455)

#### Extending classes with namedtuples
###### Methods can be added to namedtuples by extending it with a class

In [37]:
class AddCategories(Student):
    def getCategory(self):
        if self.grade == 10:
            return 'A+'
        elif self.grade > 9:
            return 'A'
        else:
            return 'B'


In [38]:
cat_1 = AddCategories(9.8,1142)
print(cat_1.getCategory())
cat_2 = AddCategories(*stud_2)
print(cat_2.getCategory())

A
B


### Logging
#### Logging is a method by which each and every line of the code can be monitored and their status saved for debugging or record keeping.
### Logging has 5 levels, in increasing order of severity,
 ###### 1. Debug
 ###### 2. Info
 ###### 3. Warning
 ###### 4. Error
 ###### 5. Crirtical
 
### Logging mode is decided by the user depending on the use, all the levels equal to or greater than the selected level will be logged. By default, the logging mode is Warning.

In [2]:
import logging
from importlib import reload
reload(logging)

<module 'logging' from '/home/sudhamsa/anaconda3/lib/python3.7/logging/__init__.py'>

In [4]:
logging.basicConfig(level=logging.ERROR,
                    filename="log-test.log", filemode="w",
                    format='%(process)d - %(levelname)s - %(asctime)s - %(message)s')
logging.debug('This is a debug message')
logging.info('This is a info message')
logging.basicConfig(level=logging.DEBUG)
logging.warning('This is a warning message', exc_info=True)
logging.error('This is an error message')
logging.critical('This is a critical message')
logging.debug('This is a debug message')

### Important
#### basicConfig method is made to run only once, so you can't change logging level anywhere in between, therefore you should use the basicConfig method before logging anything because all the logging methods, viz. logging.debug(), logging.info(), etc, call basicConfig method internally, so if anything is logged, you can't configure basicConfig() anymore until you reset the root handlers.

## PEP 8
[A Guide to PEP8](https://realpython.com/python-pep8/)

In [26]:
import this

In [1]:
import antigravity